# ResNet 18

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

data_path = "SportsImageClassification"
batch_size = 32
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Transformations ====
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  
])

train_data = datasets.ImageFolder(f"{data_path}/train", transform=transform)
valid_data = datasets.ImageFolder(f"{data_path}/valid", transform=transform)
test_data  = datasets.ImageFolder(f"{data_path}/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size)
test_loader  = DataLoader(test_data, batch_size=batch_size)

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_data.classes))  
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"[{epoch+1}/{num_epochs}] Loss: {total_loss/len(train_loader):.4f}")

model.eval()
correct = total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f" Test Accuracy: {correct/total:.2%}")


In [ ]:
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import cv2

def generate_gradcam(image, model, class_names):
    input_tensor = image.unsqueeze(0).to(device)
    model.eval()
    activations.clear()
    gradients.clear()

    output = model(input_tensor)
    pred_class = output.argmax().item()

    model.zero_grad()
    output[0, pred_class].backward()

    grad = gradients[0]
    act = activations[0]
    weights = grad.mean(dim=(2, 3), keepdim=True)
    cam = (weights * act).sum(dim=1).squeeze()
    cam = F.relu(cam)
    cam -= cam.min()
    cam /= cam.max()
    cam = cam.cpu().detach().numpy()
    cam = cv2.resize(cam, (224, 224))

    img_np = image.permute(1, 2, 0).numpy() * 0.5 + 0.5
    img_np = np.clip(img_np, 0, 1)

    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB) / 255.0
    superimposed = np.clip(0.6 * img_np + 0.4 * heatmap, 0, 1)

    return img_np, cam, superimposed, class_names[pred_class]

activations = []
gradients = []

target_layer = model.layer4[1].conv2
target_layer.register_forward_hook(lambda m, i, o: activations.append(o))
target_layer.register_full_backward_hook(lambda m, gi, go: gradients.append(go[0]))

images_to_plot = [41, 44]  

fig, axs = plt.subplots(len(images_to_plot), 3, figsize=(10, 4 * len(images_to_plot)))

for idx, image_idx in enumerate(images_to_plot):
    image, label = test_data[image_idx]
    original, cam_map, fusion, predicted_class = generate_gradcam(image, model, train_data.classes)

    axs[idx, 0].imshow(original)
    axs[idx, 0].set_title("Image originale")
    axs[idx, 1].imshow(cam_map, cmap="jet")
    axs[idx, 1].set_title("Carte Grad-CAM")
    axs[idx, 2].imshow(fusion)
    axs[idx, 2].set_title(f"Grad-CAM sur : {predicted_class}")

    for ax in axs[idx]:
        ax.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import pickle

torch.save(model.state_dict(), "resnet_model.pth")

with open("class_names.pkl", "wb") as f:
    pickle.dump(train_data.classes, f)

print(" Modèle et classes sauvegardés pour Streamlit.")


In [ ]:
from collections import Counter

print("Répartition train :", Counter([label for _, label in train_data]))
print("Répartition test  :", Counter([label for _, label in test_data]))


In [ ]:
correct_val = total_val = 0
with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        correct_val += (preds == labels).sum().item()
        total_val += labels.size(0)

print(f" Validation Accuracy: {correct_val / total_val:.2%}")


In [ ]:
train_data = datasets.ImageFolder(f"{data_path}/train", transform=transform)
valid_data = datasets.ImageFolder(f"{data_path}/valid", transform=transform)
test_data  = datasets.ImageFolder(f"{data_path}/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size)
test_loader  = DataLoader(test_data, batch_size=batch_size)

def plot_class_distribution(data, title):
    labels = [label for _, label in data]
    counter = Counter(labels)
    classes = data.classes
    dist = {classes[i]: counter[i] for i in counter}
    
    plt.figure(figsize=(12, 4))
    plt.bar(dist.keys(), dist.values())
    plt.xticks(rotation=90)
    plt.title(title)
    plt.tight_layout()
    plt.show()

    total = sum(dist.values())
    top3 = sorted(dist.items(), key=lambda x: x[1], reverse=True)[:3]
    bottom3 = sorted(dist.items(), key=lambda x: x[1])[:3]

    print(f" Nombre total d'images : {total}")
    print(" Top 3 des classes les plus représentées :")
    for cls, count in top3:
        print(f"   - {cls} : {count} images")
    print(" Top 3 des classes les moins représentées :")
    for cls, count in bottom3:
        print(f"   - {cls} : {count} images")

        
print(" Répartition des classes (train)")
plot_class_distribution(train_data, "Répartition des classes (train)")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

cm = confusion_matrix(y_true, y_pred, labels=list(range(len(test_data.classes))))
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

accuracies = np.diag(cm_normalized)
worst_indices = np.argsort(accuracies)[:10]  

cm_worst = cm_normalized[np.ix_(worst_indices, worst_indices)]
labels_worst = [test_data.classes[i] for i in worst_indices]

plt.figure(figsize=(10, 8))
sns.heatmap(cm_worst, annot=True, fmt=".2f", xticklabels=labels_worst, yticklabels=labels_worst, cmap="Reds")
plt.title("Matrice de confusion — Top 10 classes les moins bien prédites")
plt.xlabel("Classe prédite")
plt.ylabel("Classe réelle")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

model.eval()
errors = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        for img, label, pred in zip(images, labels, preds):
            if label != pred:
                errors.append((img.cpu(), label.item(), pred.item()))

fig, axs = plt.subplots(3, 3, figsize=(10, 10))
for ax, (img, label, pred) in zip(axs.flat, errors[:9]):
    ax.imshow(img.permute(1, 2, 0) * 0.5 + 0.5)  
    ax.set_title(f"True: {test_data.classes[label]}\nPred: {test_data.classes[pred]}")
    ax.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
y_true = np.array(y_true)
y_pred = np.array(y_pred)
per_class_accuracy = {}

for i, class_name in enumerate(test_data.classes):
    mask = y_true == i
    correct = (y_pred[mask] == i).sum()
    total = mask.sum()
    acc = correct / total if total > 0 else 0
    per_class_accuracy[class_name] = acc

sorted_acc = sorted(per_class_accuracy.items(), key=lambda x: x[1])
for cls, acc in sorted_acc[:10]:
    print(f"{cls:25s} : {acc:.2%}")


# Comparaison de différents modèles

In [24]:
from sklearn.metrics import accuracy_score, f1_score
import time
import copy
import os


In [17]:
def load_model(model_name, num_classes):
    if model_name == "resnet18":
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == "vgg16":
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    elif model_name == "efficientnet_b0":
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError(f"Modèle {model_name} non supporté.")

    return model.to(device)


In [18]:
def train_model(model, train_loader, num_epochs=5, lr=1e-4):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    history = []

    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"[{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f}")
        history.append(avg_loss)

    train_time = time.time() - start_time
    return model, history, train_time


In [19]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu()
            all_preds.extend(preds)
            all_labels.extend(labels)

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return acc, f1


In [ ]:
os.makedirs("saved_models", exist_ok=True)

models_to_test = ["efficientnet_b0", "vgg16", "resnet18"]

for name in models_to_test:
    print(f"\n=== 🔍 Testing {name.upper()} ===")

    model = load_model(name, num_classes=len(train_data.classes))
    model, loss_hist, train_time = train_model(model, train_loader, num_epochs=num_epochs)
    acc, f1 = evaluate_model(model, test_loader)

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f" Accuracy: {acc:.2%}")
    print(f" F1-score: {f1:.4f}")
    print(f" Train Time: {train_time:.2f}s")
    print(f" Parameters: {total_params:,}")

    model_path = f"saved_models/{name}_model.pth"
    class_path = f"saved_models/{name}_classes.pkl"

    torch.save(model.state_dict(), model_path)
    with open(class_path, "wb") as f:
        pickle.dump(train_data.classes, f)

    print(f"Modèle sauvegardé sous {model_path}")
    print(f"Classes sauvegardées sous {class_path}")

# Overfitting check on ResNet 18

In [25]:
def train_model(model, train_loader, val_loader, optimizer, num_epochs=5, lr=1e-4, verbose=True):
    criterion = nn.CrossEntropyLoss()
    train_losses = []
    val_losses = []
    val_accuracies = []

    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_acc = correct / total
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_acc)

        if verbose:
            print(f"[{epoch+1}/{num_epochs}] 🏋️ Train Loss: {avg_train_loss:.4f} | 🧪 Val Loss: {avg_val_loss:.4f} | 🎯 Val Acc: {val_acc:.2%}")

    train_time = time.time() - start_time
    return model, train_losses, val_losses, val_accuracies, train_time


In [26]:
def plot_training_curves(train_losses, val_losses, val_accuracies):
    epochs = range(1, len(train_losses)+1)

    plt.figure(figsize=(12,4))

    plt.subplot(1,2,1)
    plt.plot(epochs, train_losses, label="Train Loss")
    plt.plot(epochs, val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Train vs Val Loss")
    plt.legend()
    plt.grid(True)

    plt.subplot(1,2,2)
    plt.plot(epochs, val_accuracies, label="Val Accuracy", color='green')
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Validation Accuracy")
    plt.ylim(0,1)
    plt.legend()
    plt.grid(True)

    plt.show()


In [ ]:
model = load_model("resnet18", num_classes=len(train_data.classes))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

model, train_losses, val_losses, val_accuracies, train_time = train_model(
    model, train_loader, valid_loader, optimizer, num_epochs=5
)

In [ ]:
plot_training_curves(train_losses, val_losses, val_accuracies)